In [1]:
import os
import eunomia
import pickle


C:\Users\mehra\anaconda3\envs\elm\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [27]:
paper_id = '29'
docs_processor = eunomia.LoadDoc('../Data/SI_PDF/', paper_id=paper_id)
sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments'],
                                             chunk_size=None, chunk_overlap=0)

In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

k = 8
Embedding_model = 'text-davinci-003' # text-embedding-ada-002

faiss_index_path = f"../Data/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
just_load = True
if os.path.isfile(faiss_index_path) and just_load:
    print('loadded Embeddings')
    with open(faiss_index_path, "rb") as f:
        faiss_index = pickle.load(f)
else:
    with open(faiss_index_path, "wb") as f:  # text-embedding-ada-002
        faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))    # 'text-davinci-003
        pickle.dump(faiss_index, f)
retriever  = faiss_index.as_retriever(search_type="mmr", search_kwargs={"k":k}) # Maximum Marginal Relevance Retrieval


loadded Embeddings


In [4]:
from langchain.llms import OpenAI
from langchain.chat_models  import ChatOpenAI
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-16k') # gpt-3.5-turbo-16k
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False,)

C:\Users\mehra\anaconda3\envs\elm\lib\site-packages\langchain\llms\openai.py:173: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\mehra\anaconda3\envs\elm\lib\site-packages\langchain\llms\openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
query = 'What is DOI, paper title, published journal?'

from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    result = qa({"query": query})
    print(result)
    print(cb)

{'query': 'What is DOI, paper title, published journal?', 'result': 'The DOI (Digital Object Identifier) is 10.1038/ncomms1956. The paper title is "Microporous metal-organic framework with potential for carbon dioxide capture at ambient conditions". The published journal is Nature Communications.'}
Tokens Used: 4798
	Prompt Tokens: 4750
	Completion Tokens: 48
Successful Requests: 1
Total Cost (USD): $0.014442


In [6]:
Rule = f"""

There are only 2 options for water stability: \
1. Stable \
2. Unstable \

For 1. Stable, \
The MOF should be stable to keep structure integrity \
after exposure to aqueous solution. \

For 2. Unstable, \
The MOF will decompose after exposure \
to humid environment. \

"""

In [7]:
prompt3a = f"""
    This doc describes the water stability properties of afew \
    Metal-Organic Framework(MOF) compound. Please find \
    the full name for all MOFs, and the \
    water stability corresponding to each MOF. \
    Please use the following rules to identify\
    the water stability of each MOF. \

    Rule: ```{Rule}``` \

    If there are multiple MOFs, please list them all and put "MOF name" before them. \

    You should only respond in JSON format as described below. 
    
    Make sure it is parsable using Python json.load.
    
    Make sure "MOF name"s do not have duplicates

    If information is not provided, state it as not provided.\

    """

from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    result = qa({"query": prompt3a})
    print(result)
    print(cb)

{'query': '\n    This doc describes the water stability properties of afew     Metal-Organic Framework(MOF) compound. Please find     the full name for all MOFs, and the     water stability corresponding to each MOF.     Please use the following rules to identify    the water stability of each MOF. \n    Rule: ```\n\nThere are only 2 options for water stability: 1. Stable 2. Unstable \nFor 1. Stable, The MOF should be stable to keep structure integrity after exposure to aqueous solution. \nFor 2. Unstable, The MOF will decompose after exposure to humid environment. \n``` \n    If there are multiple MOFs, please list them all and put "MOF name" before them. \n    You should only respond in JSON format as described below. \n    \n    Make sure it is parsable using Python json.load.\n    \n    Make sure "MOF name"s do not have duplicates\n\n    If information is not provided, state it as not provided.\n    ', 'result': '{\n  "MOF name": "UTSA-16",\n  "Water Stability": "Stable"\n}'}
Token

In [8]:
from typing import List
from pydantic import BaseModel, Field, validator, ValidationError
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
# def parse_to_dict(result, paper_id):

#     class MOF(BaseModel):
#         name: str = Field(description="name of a MOF")
#         stability: str = Field(description="choose only one: stable or unstable or not provided")

#         def name_not_provided(cls, field):
#             if field == "Not Provided":
# #                 raise ValueError("Dummy-MOF!")
#                 field = 'Dummy-MOF'
#             return field
    
#     class MOFList(BaseModel):
#         MOFs: List[MOF]

#     from langchain.output_parsers import PydanticOutputParser
#     parser = PydanticOutputParser(pydantic_object=MOF)
#     llm_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(temperature=0))  
    
# #     print(result['result'])
#     parsed_result = llm_parser.parse(result['result'].strip("'").replace("\\", ""))
#     dict_response = dict(zip(parsed_result.name, parsed_result.stability))
#     new_dict = {}
#     for i, (key, value) in enumerate(dict_response.items(), start=1):
#         new_dict[key] = {'Predicted Stability': value, 'Paper id': paper_id}
#     return new_dict

def parse_to_dict(result, paper_id):
    from typing import List
    from langchain.output_parsers import PydanticOutputParser

    class MOF(BaseModel):
        name: str = Field(description="name of a MOF")
        stability: str = Field(description="choose only one: stable or unstable or not provided")

    class MOFList(BaseModel):
        MOFs: List[MOF]

    parser = PydanticOutputParser(pydantic_object=MOFList)
    llm_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(temperature=0))

    try:
        parsed_result = llm_parser.parse(result['result'])
    except OutputParserException as e:
        raise OutputParserException(f"Failed to parse MOF: {e}", llm_output=result['result'])

    dict_response = {mof.name: mof.stability for mof in parsed_result.MOFs}
    new_dict = {key: {'Predicted Stability': value, 'Paper id': paper_id} for key, value in dict_response.items()}
    return new_dict

In [9]:
predictions_dict = parse_to_dict(result, paper_id)
predictions_dict

{'UTSA-16': {'Predicted Stability': 'stable', 'Paper id': '69'}}

In [10]:
import os
import pickle
from langchain.callbacks import get_openai_callback
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models  import ChatOpenAI


def retriev_QA(paper_id, LLM_model, Embedding_model, k, temperature):
    mofpaper_path = '../Data/SI_PDF/' + paper_id + '.pdf'
    print(mofpaper_path)
    with get_openai_callback() as cb:
        docs_processor = eunomia.LoadDoc('../Data/SI_PDF/', paper_id=str(paper_id))
        sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments'],
                                                     chunk_size=None, chunk_overlap=0)
        print(len(docs_processor.pages), len(sliced_pages))
        faiss_index_path = f"../Data/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
        if os.path.isfile(faiss_index_path):
            with open(faiss_index_path, "rb") as f:
                faiss_index = pickle.load(f)
        else:
            with open(faiss_index_path, "wb") as f:
                faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))   #text-davinci-003
                pickle.dump(faiss_index, f)
        retriever  = faiss_index.as_retriever(search_type="mmr", search_kwargs={"k":k}) # Maximum Marginal Relevance Retrieval        
#         retriever  = faiss_index.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .8}) # Maximum Marginal Relevance Retrieval        
        qa = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(model=LLM_model, temperature=temperature), chain_type="stuff", retriever=retriever, return_source_documents=False)
        answer = qa({"query": prompt3a})
        predictions_dict = parse_to_dict(answer, paper_id)
        return predictions_dict, cb.total_cost

In [16]:
from tqdm import tqdm
import numpy as np
import json
predictions = []
costs = []


LLM_model = 'gpt-3.5-turbo-16k'
Embedding_model = 'text-davinci-003'#'text-davinci-003' text-embedding-ada-002
k = 8
temp = 0.9

for i in tqdm(range(1,112)):
    print(f'Paper {i}')
    p, c = retriev_QA(str(i), LLM_model, Embedding_model, k, temperature=temp)
    costs.append(c)
    predictions.append(p)
    print(f'\n Total cost: ${np.sum(costs)}\n') 
    results_index_path = f"../Data/faiss/results/{LLM_model}/faiss-{Embedding_model}/k_{k}/t_{temp}/results_paper_{i}.json"
    with open(results_index_path, 'w') as f:
        json.dump(p, f)
#     break

  0%|                                                                                                                                                                                 | 0/111 [00:00<?, ?it/s]

Paper 1
../Data/SI_PDF/1.pdf
15 13


  1%|█▌                                                                                                                                                                       | 1/111 [00:07<13:53,  7.57s/it]


 Total cost: $0.01808

Paper 2
../Data/SI_PDF/2.pdf
15 15


  2%|███                                                                                                                                                                      | 2/111 [00:18<16:59,  9.35s/it]

...


 Total cost: $2.0593295

Paper 110
../Data/SI_PDF/110.pdf
13 13


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 110/111 [15:34<00:05,  5.90s/it]


 Total cost: $2.0843760000000002

Paper 111
../Data/SI_PDF/111.pdf
8 8


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [15:39<00:00,  8.46s/it]


 Total cost: $2.1057175000000004



## Two-step LLM

In [24]:
chain_1_prompt = f"""
    This doc describes the water stability properties of a few \
    Metal-Organic Framework(MOF) compound. Please find \
    the full name for all MOFs.
    Please use the following rules:

    Rule: \

    If there are multiple MOFs, please list them all and put "MOF name" before them. \

    Your output sould be a Python list.
    
    Make sure "MOF name"s do not have duplicates.

    """




def two_step_retriev_QA_(paper_id, LLM_model, Embedding_model, k, temperature):
    mofpaper_path = '../Data/SI_PDF/' + paper_id + '.pdf'
    print(mofpaper_path)
    with get_openai_callback() as cb:
        docs_processor = eunomia.LoadDoc('../Data/SI_PDF/', paper_id=str(paper_id))
        sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments'],
                                                     chunk_size=None, chunk_overlap=0)
        print(len(docs_processor.pages), len(sliced_pages))
        faiss_index_path = f"../Data/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
        if os.path.isfile(faiss_index_path):
            with open(faiss_index_path, "rb") as f:
                faiss_index = pickle.load(f)
        else:
            with open(faiss_index_path, "wb") as f:
                faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))   #text-davinci-003
                pickle.dump(faiss_index, f)
        retriever  = faiss_index.as_retriever(search_type="mmr", search_kwargs={"k":k}) # Maximum Marginal Relevance Retrieval        
#         retriever  = faiss_index.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .8}) # Maximum Marginal Relevance Retrieval        
        qa = RetrievalQA.from_chain_type(
            llm=ChatOpenAI(model=LLM_model, temperature=temperature), chain_type="stuff", retriever=retriever, return_source_documents=False)
        answer_1 = qa({"query": chain_1_prompt})
        mof_list = answer_1['result']
        chain_2_prompt = f"""
        This doc describes the water stability properties of a few \
        Metal-Organic Framework(MOF) compound listed as {mof_list}. Please find \
        the water stability corresponding to each MOF. \
        Please use the following rules to identify\
        the water stability of each MOF. \

        Rule: \
        There are only 2 options for water stability: \
        1. Stable \
        2. Unstable \

        For 1. Stable, \
        The MOF should be stable to keep structure integrity \
        after exposure to aqueous solution. \

        For 2. Unstable, \
        The MOF will decompose after exposure \
        to humid environment. \


        If there are multiple MOFs, please list them all and put "MOF name" before them. \

        You should only respond in JSON format as described below. \

        Make sure it is parsable using Python json.load.\

        Make sure "MOF name"s do not have duplicates.\

        If information is not provided, state it as not provided.\


        """
        answer_2 = qa(chain_2_prompt)
        predictions_dict = parse_to_dict(answer_2, paper_id)
        return predictions_dict, cb.total_cost

In [28]:
predictions = []
costs = []

LLM_model = 'gpt-3.5-turbo-16k'
Embedding_model = 'text-davinci-003'#'text-davinci-003' text-embedding-ada-002
k = 8
temp = 0

for i in tqdm(range(29,112)):
    print(f'Paper {i}')
    p, c = two_step_retriev_QA_(str(i), LLM_model, Embedding_model, k, temperature=temp)
    costs.append(c)
    predictions.append(p)
    print(f'\n Total cost: ${np.sum(costs)}\n') 
    results_index_path = f"../Data/faiss/results/{LLM_model}/faiss-{Embedding_model}/k_{k}/t_{temp}/two-step/results_paper_{i}.json"
    with open(results_index_path, 'w') as f:
        json.dump(p, f)
#     break

  0%|                                                                                                                                                                                  | 0/83 [00:00<?, ?it/s]

Paper 29
../Data/SI_PDF/29.pdf
13 13


  1%|██                                                                                                                                                                        | 1/83 [00:21<29:33, 21.63s/it]


 Total cost: $0.05010850000000001

Paper 30
../Data/SI_PDF/30.pdf
9 7


  2%|████                                                                                                                                                                      | 2/83 [00:26<16:01, 11.87s/it]

...


 Total cost: $2.911996

Paper 110
../Data/SI_PDF/110.pdf
13 13


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉  | 82/83 [13:16<00:08,  8.32s/it]


 Total cost: $2.9606005

Paper 111
../Data/SI_PDF/111.pdf
8 8


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83/83 [13:24<00:00,  9.70s/it]


 Total cost: $3.00245



In [ ]:
from typing import List
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser

class MOF(BaseModel):
    name: List[str] = Field(description="name of a MOF")
    stability: List[str] = Field(description="choose only one: stable or unstable or not provided")
    @validator("name", pre=True)
    def name_not_provided(cls, field):
        if field == "Not Provided":
#                 raise ValueError("Dummy-MOF!")
            field = 'Dummy-MOF'
        return field

from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=MOF)
#     try:
llm_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(temperature=0))

test_dict = {
  "MOFs": [
    {
      "MOF name": "DMOF",
      "Water stability": "Stable"
    },
    {
      "MOF name": "MOF-508",
      "Water stability": "Stable"
    },
    {
      "MOF name": "DMOF-TM",
      "Water stability": "Not provided"
    },
    {
      "MOF name": "MOF-508-TM",
      "Water stability": "Not provided"
    }
  ]
}

In [ ]:
parsed_result = parse_to_dict(test_dict,65)